## Regression Model for USDA Beef Grading

This model will use pytoch and resnet to make a CNN regression model (convolutional neural network)
hay que mantener 5 pasos
1. cargando imagenes y procesarlos
2. modificar el modelo resnet para la regresion
3. entrenar el model
4. evaluar modelo (root mean square)

In [15]:
import torch
import torchvision
import os
import torch.nn as nn
import numpy as np
from torchvision import datasets, transforms, models
import torch.optim as optim
import pandas as pd
from torchvision.models import ResNet18_Weights
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR

from PIL import Image


from sklearn.metrics import mean_squared_error

print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.4


In [16]:
root_folder =  "Data/Cleaned JPEG Images"
training_folders = [
    "Cargill Dodge 07 20 22 Plant", 
    "Cargill Fort Morgan 03 28 22 Plant",
    "Cargill Schuyler 04 01 22 Plant",
    "Cargill Fort Morgan 07 15 22 Plant"
]

testing_folders= [
    "Cargill Friona 07 18 22 Plant",
    "Cargill Schuyler 07 18 22 Plant",
    "Cargill Fort Morgan 05 12 22 Plant",
]
#df = pd.read_excel(excel_path)

#df.columns = df.columns.str.strip()
#ID_to_score = dict(zip(df['CarcassId'], df['Average']))
#ID_to_score = {f"{carcass_id}.jpg": avg for carcass_id, avg in zip(df['ID'], df['AVG'])}

#print(ID_to_score.keys())

def DataPrepare(root_folder, input_folders):
    all_image_addresses= []
    all_Scores = []
    total_image_number = 0
    for i in range((len(input_folders))):
        img_folder_name = root_folder + "/" +  input_folders[i] + "/Images/"
        print(img_folder_name)
        Scores_name = root_folder + "/" + input_folders[i] + "/data.xlsx"
        print(Scores_name)
        df = pd.read_excel(Scores_name)
        for _, _, files in os.walk(img_folder_name):
            for filename in files:
                total_image_number += 1
                all_image_addresses.append(img_folder_name + filename)
                Score_value = df.at[int(df.index[df['ID'] == filename[0:-4]][0]), 'AVG']
                all_Scores.append(Score_value)
    return all_image_addresses, all_Scores, total_image_number

class ImageDataset(Dataset):
    def __init__(self, root_folder, input_folders, transform =None):
        self.img_dirs, self.scores, self.img_num = DataPrepare(root_folder, input_folders)
        self.transform = transform
                
    def __len__(self):
        return self.img_num
        
    def __getitem__(self, idx):
        image = Image.open(self.img_dirs[idx])
        if self.transform:
            image = self.transform(image)
        score = self.scores[idx] / 1500
        return image, score
      

# Transforms and model

In [17]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])

traindataset = ImageDataset(root_folder, training_folders, transform=transform)
dataloader= DataLoader(traindataset, batch_size=32, shuffle=True)

Data/Cleaned JPEG Images/Cargill Dodge 07 20 22 Plant/Images/
Data/Cleaned JPEG Images/Cargill Dodge 07 20 22 Plant/data.xlsx
Data/Cleaned JPEG Images/Cargill Fort Morgan 03 28 22 Plant/Images/
Data/Cleaned JPEG Images/Cargill Fort Morgan 03 28 22 Plant/data.xlsx
Data/Cleaned JPEG Images/Cargill Schuyler 04 01 22 Plant/Images/
Data/Cleaned JPEG Images/Cargill Schuyler 04 01 22 Plant/data.xlsx
Data/Cleaned JPEG Images/Cargill Fort Morgan 07 15 22 Plant/Images/
Data/Cleaned JPEG Images/Cargill Fort Morgan 07 15 22 Plant/data.xlsx


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [13]:
   #load pretrained resnet model
resnet = models.resnet18(weights=ResNet18_Weights.DEFAULT)

num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features,1) #output 1 for regression

#LOAD IN THE GPU (MUY IMPORTANTE NO OLVIDES)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
resnet = resnet.to(device)

cuda


# Training Loop

In [14]:
criterion = nn.MSELoss()
optimizer = optim.Adam(resnet.parameters(), lr = 0.001)
scheduler = StepLR(optimizer, step_size=40, gamma=0.1)

num_epochs = 65

for epoch in range(num_epochs):
    running_loss =0.0
    for images, scores in dataloader:
        images = images.to(device)
        scores = scores.to(device).view(-1,1).float()
        
        optimizer.zero_grad()
        
        outputs = resnet(images)
        loss = criterion(outputs, scores)
        
        #backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    scheduler.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')
    
print('Training completed!')

NameError: name 'dataloader' is not defined

# Evaluation

In [174]:
def evaluate_model(model, dataloader):
    model.eval()
    all_preds = []
    all_scores = []
    with torch.no_grad():
        for images, scores in dataloader:
            images = images.to(device)
            scores = scores.to(device).view(-1,1).float()
            outputs = model(images)
            all_preds.append(outputs.cpu().numpy())
            all_scores.append(scores.cpu().numpy())
            
    all_preds = np.concatenate(all_preds)
    all_scores = np.concatenate(all_scores)
    mse = mean_squared_error(all_scores, all_preds)
    rmse = np.sqrt(mse)
    
    print(f'RMSE: {rmse:.4f}')
    return rmse

evaluate_model(resnet, dataloader)

RMSE: 31.2301


31.230062

# Misc / Junk Code